## Homework3

下記の「#common」で始まるセルの中で指定されたリンク先にある購買データをベースに以下の処理を実行し、問いに答えて課題を提出してください。（データの前処理は、以下の記載の通りに行ってください。）
    
処理内容

1. `#common`のセルで事前に前処理されたデータ`target_online_retail_data_tb`を用います。
2. 購買者（`CustomerID`）の各合計購買金額（`TotalPrice`の合計）を算出し、さらに合計購買金額をベースに降順に並び替えをします。  
3. 合計購買金額の降順に並んだ購買者を$n$等分（$n$はデータ数を越えず、分位数に同一の値が存在しない自然数。購買者数が$n$で割り切れるとは限らず、この場合の処理は`Pandas.qcut`の処理に準じます。）にグループ分けします。  その際、合計購買金額が大きいグループから順に「グループ1」「グループ2」$\cdots$「グループn」とします。
4. それぞれのグループの金額合計値を算出してください（このアプローチをデシル分析といいます）。
5. それぞれのグループが売上の何割を占めるかを求めてください。その際、「グループ1」〜「グループn」の順に並べ、`Pandas.Series`形式で回答してください。


(なお、マーケティング戦略では、このように顧客を分けることをセグメンテーションといい、上位2割に絞ってアプローチを仕掛けることをターゲティングといいます。それぞれの戦略によりますが、優良顧客に的を絞った方が投資対効果が高いことが多いため、このようなアプローチを取ることがあります。これに関連して、上位2割の顧客が売上全体の8割を占めることを「パレートの法則」と言います。)

提出するときは、以下の点に注意してください。  
>- 以下の関数`homework`の`!!WRITE ME!!`に処理を書いてください。(**「`!!WRITE ME!!`」は消して、記入してください。**)
>- 実際の提出は記述された`homework`関数全てになり、**提出はOmnicampus内の宿題の欄から今週の課題を選択後、提出内容に関数を貼り付けてから[Pythonコード提出]を押してください。**
>- 前処理は以下のコードを使ってください。  
>- 返り値が`Pandas.Series`形式になるようにしてください。  
>- 解答は、`my_result=pd.Series([0.6,0.2,0.1,0.025,0.025,0.022,0.021,0.02,0.01,0]) `のような形になります。(売上割合はグループ順にして、合計は1になるようにします。)

In [ ]:
# common
import pandas as pd
import numpy as np

# init part(データの読み込みと前処理)
file_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
online_retail_data = pd.ExcelFile(file_url)
online_retail_data_table = online_retail_data.parse('Online Retail')

# 採点の都合上、文字列型に変換
online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(lambda x:str(x)[0])

# InvoiceNoの先頭が5であるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5') 
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)

In [ ]:
target_online_retail_data_tb.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34


In [ ]:
print(target_online_retail_data_tb["Quantity"].shape)
print(target_online_retail_data_tb["UnitPrice"].shape)

(397924,)
(397924,)


In [ ]:
# 欠損のチェック
target_online_retail_data_tb.isnull().sum() 

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
cancel_flg     0
TotalPrice     0
dtype: int64

In [ ]:
target_online_retail_data_tb.describe()

,Quantity,UnitPrice,CustomerID,TotalPrice
count,397924.000000,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171,22.394749
std,180.420210,22.096788,1713.169877,309.055588
min,1.000000,0.000000,12346.000000,0.000000
25%,2.000000,1.250000,13969.000000,4.680000
50%,6.000000,1.950000,15159.000000,11.800000
75%,12.000000,3.750000,16795.000000,19.800000
max,80995.000000,8142.750000,18287.000000,168469.600000


In [ ]:
TotalPrice = target_online_retail_data_tb["Quantity"] * target_online_retail_data_tb["UnitPrice"]
df1 = pd.DataFrame(data=target_online_retail_data_tb["CustomerID"])
df1["TotalPrice"] = TotalPrice
df1_sort = df1.sort_values(by="TotalPrice", ascending=False).reset_index()
del df1_sort["index"]
print(df1_sort)
#df1_parties = df1_sort.quantile(q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).to_dict()
#print(df1_parties)

        CustomerID  TotalPrice
0          16446.0   168469.60
1          12346.0    77183.60
2          15098.0    38970.00
3          16029.0     8142.75
4          17450.0     7144.72
...            ...         ...
397919     16818.0        0.00
397920     15581.0        0.00
397921     12446.0        0.00
397922     12431.0        0.00
397923     13081.0        0.00

[397924 rows x 2 columns]


In [ ]:
n = 2
df1_sort_qcut = pd.qcut(df1_sort.TotalPrice,n)
pd.value_counts(df1_sort_qcut)
df1_sort_qcut

0         (11.8, 168469.6]
1         (11.8, 168469.6]
2         (11.8, 168469.6]
3         (11.8, 168469.6]
4         (11.8, 168469.6]
                ...       
397919      (-0.001, 11.8]
397920      (-0.001, 11.8]
397921      (-0.001, 11.8]
397922      (-0.001, 11.8]
397923      (-0.001, 11.8]
Name: TotalPrice, Length: 397924, dtype: category
Categories (2, interval[float64, right]): [(-0.001, 11.8] < (11.8, 168469.6]]

In [ ]:
df1_sort_qcut.sum(level='key2')

TypeError: ignored

In [ ]:
# 分位点で分ける関数を定義
def cal_decil(x, col, df):
    if x <= df[col][0.1]:
        return 1
    elif x <= df[col][0.2]:
        return 2
    elif x <= df[col][0.3]: 
        return 3
    elif x <= df[col][0.4]: 
        return 4
    elif x <= df[col][0.5]: 
        return 5
    elif x <= df[col][0.6]: 
        return 6
    elif x <= df[col][0.7]: 
        return 7
    elif x <= df[col][0.8]: 
        return 8
    elif x <= df[col][0.9]: 
        return 9
    else:
        return 10

# デシルスコアの算出
df1_sort['Decil_score'] = df1_sort['TotalPrice'].apply(cal_decil, args=('TotalPrice', df1_parties))
df1_sort.head(100000)

,CustomerID,TotalPrice,Decil_score
0,16446.0,168469.60,10
1,12346.0,77183.60,10
2,15098.0,38970.00,10
3,16029.0,8142.75,10
4,17450.0,7144.72,10
...,...,...,...
99995,17725.0,19.80,8
99996,16143.0,19.80,8
99997,16305.0,19.80,8
99998,17725.0,19.80,8


In [ ]:
df1_sort['Cumsum'] = df1_sort['TotalPrice'].cumsum()
df1_sort['Cumprod'] = df1_sort['Cumsum'] / df1_sort['Cumsum'].max()
df1_sort

,CustomerID,TotalPrice,Decil_score,Cumsum,Cumprod
0,16446.0,168469.60,10,1.684696e+05,0.018905
1,12346.0,77183.60,10,2.456532e+05,0.027566
2,15098.0,38970.00,10,2.846232e+05,0.031939
3,16029.0,8142.75,10,2.927660e+05,0.032853
4,17450.0,7144.72,10,2.999107e+05,0.033655
...,...,...,...,...,...
397919,16818.0,0.00,1,8.911408e+06,1.000000
397920,15581.0,0.00,1,8.911408e+06,1.000000
397921,12446.0,0.00,1,8.911408e+06,1.000000
397922,12431.0,0.00,1,8.911408e+06,1.000000


In [ ]:
# 各階層の売り上げが占める割合
df = df1_sort[['Decil_score', 'Cumprod']].groupby('Decil_score').max() \
    .reset_index().sort_values('Decil_score', ascending=False)
df

,Decil_score,Cumprod
9,10,0.539904
8,9,0.657830
7,8,0.746627
6,7,0.809900
5,6,0.884251
4,5,0.928957
3,4,0.961541
2,3,0.980253
1,2,0.994457
0,1,1.000000


In [ ]:
groupname = ["グループ1","グループ2","グループ3","グループ4","グループ5","グループ6","グループ7","グループ8","グループ9","グループ10"]
df["groupname"] = groupname
print(df)
s = df['Cumprod']
print(type(s))

   Decil_score   Cumprod groupname
9           10  0.539904     グループ1
8            9  0.657830     グループ2
7            8  0.746627     グループ3
6            7  0.809900     グループ4
5            6  0.884251     グループ5
4            5  0.928957     グループ6
3            4  0.961541     グループ7
2            3  0.980253     グループ8
1            2  0.994457     グループ9
0            1  1.000000    グループ10
<class 'pandas.core.series.Series'>


In [ ]:
# 分位点で分ける関数を定義
def cal_decil(x, col, df):
    if x <= df[col][0.1]:
        return 1
    elif x <= df[col][0.2]:
        return 2
    elif x <= df[col][0.3]: 
        return 3
    elif x <= df[col][0.4]: 
        return 4
    elif x <= df[col][0.5]: 
        return 5
    elif x <= df[col][0.6]: 
        return 6
    elif x <= df[col][0.7]: 
        return 7
    elif x <= df[col][0.8]: 
        return 8
    elif x <= df[col][0.9]: 
        return 9
    else:
        return 10

# working place. everything 
def homework(target_online_retail_data_tb, n):  
    TotalPrice = target_online_retail_data_tb["Quantity"] * target_online_retail_data_tb["UnitPrice"]
    df1 = pd.DataFrame(data=target_online_retail_data_tb["CustomerID"])
    df1["TotalPrice"] = TotalPrice
    df1_sort = df1.sort_values(by="TotalPrice", ascending=False).reset_index()
    del df1_sort["index"]
    df1_parties = df1_sort.quantile(q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).to_dict()
    # デシルスコアの算出
    df1_sort['Decil_score'] = df1_sort['TotalPrice'].apply(cal_decil, args=('TotalPrice', df1_parties))
    df1_sort['Cumsum'] = df1_sort['TotalPrice'].cumsum()
    df1_sort['Cumprod'] = df1_sort['Cumsum'] / df1_sort['Cumsum'].max()
    # 各階層の売り上げが占める割合
    df = df1_sort[['Decil_score', 'Cumprod']].groupby('Decil_score').max() \
        .reset_index().sort_values('Decil_score', ascending=False)
    groupname = ["グループ1","グループ2","グループ3","グループ4","グループ5","グループ6","グループ7","グループ8","グループ9","グループ10"]
    df["groupname"] = groupname
    s = df['Cumprod']
    return s

n = 5
homework(target_online_retail_data_tb, n)

9    0.539904
8    0.657830
7    0.746627
6    0.809900
5    0.884251
4    0.928957
3    0.961541
2    0.980253
1    0.994457
0    1.000000
Name: Cumprod, dtype: float64

In [ ]:
def homework(target_online_retail_data_tb, n):  
  df1 = pd.DataFrame(target_online_retail_data_tb.groupby(["CustomerID"])["TotalPrice"].sum().reset_index()) #df.reset_index()でインデックスを振りなおす
  df1_sort = df1.sort_values(by="TotalPrice", ascending=False)
  df1_sort["sort"] = pd.qcut(df1_sort.loc[:,"TotalPrice"],n)
  df1_cut = (pd.DataFrame(df1_sort.groupby(["sort"])["TotalPrice"].sum().reset_index())).sort_values(by="sort",ascending = False) #groupbyメソッドにas_index = Falseパラメータを設定するとインデックスが設定されなくなり、テーブルとして扱いたいときに便利
  my_result = df1_cut["TotalPrice"]/df1_cut["TotalPrice"].sum()
  return my_result
homework(target_online_retail_data_tb, 10)

# df.index.names =['key1','key2']でindexに名前を付けれる(カラム名はdf.columns.names =['city','color'])

9    0.539904
8    0.117925
7    0.088797
6    0.063273
5    0.074351
4    0.044705
3    0.032584
2    0.018712
1    0.014204
0    0.005543
Name: TotalPrice, dtype: float64

**謝辞**：以下のデータセットの利用に関して  
 https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

引用元：Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [[http://archive.ics.uci.edu/ml](http://archive.ics.uci.edu/ml)]. Irvine, CA: University of California, School of Information and Computer Science.
 
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).